# Master thesis stuff

Import necessary packages and modules.

In [1]:
import numpy as np
import scipy as sp
import scipy.optimize
import matplotlib.pyplot as plt
import tensorflow as tf
import csv

import graph


Define FVM solver.

In [2]:
class QuantumGraphSolverFVM(object):

    def __init__(self, graph):
        self.graph = graph

    def solve(self, nx, nt, eta=1):

        self.nx = nx

        # Abbreviate function f
        f = self.graph.f
        eps = self.graph.eps

        nxi = nx - 2
        n_edges = self.graph.ne
        n_nodes = self.graph.n_v

        n_inner = nxi * n_edges
        n_dofs = n_inner + n_nodes

        # Model parameters
        L = self.graph.ub[1] - self.graph.lb[1]
        self.L = L
        he = L / nx

        idxi = np.arange(nxi)
        # idxn = np.arange(self.graph.n_v)
        tau = (self.graph.ub[0] - self.graph.lb[0]) / nt

        # Number of edges adjacent to vertex
        nout = np.array([len(x) for x in self.graph.Vout])
        nin = np.array([len(x) for x in self.graph.Vin])
        n_adj = nout + nin

        def get_u_edge(u, edgeidx, timeidx=-1):
            if timeidx == -1:
                uk = u
            else:
                uk = u[:, timeidx]

            i = edgeidx

            idx_in = n_inner + self.graph.E[i][0]
            idx_inner = i * nxi + idxi
            idx_out = n_inner + self.graph.E[i][1]

            # All entries on the edge
            u_ = np.hstack((uk[idx_in:idx_in + 1],
                            uk[idx_inner],
                            uk[idx_out:idx_out + 1]))

            return u_

        self.get_u_edge = get_u_edge

        e = np.ones(nxi)
        SblkII = []
        SblkIV = []
        SblkVV = []

        Mdiag = []

        for k in range(self.graph.ne):

            # This could be outside as long as length in constant
            M_II = sp.sparse.dia_matrix(
                (e * he, np.array([0])), shape=(nxi, nxi))
            D_II = sp.sparse.dia_matrix(
                (np.array([-e, 2 * e, -e]) / he, np.array([-1, 0, 1])),
                shape=(nxi, nxi))
            ek = self.graph.E[k]
            D_IV = sp.sparse.csr_matrix((np.array(
                [-1 / he, -1 / he]),
                (np.array([0, nxi - 1]), np.array(ek))),
                shape=(nxi, self.graph.n_v))
            S_II = M_II + tau * eps * D_II
            S_IV = tau * eps * D_IV

            SblkII.append(S_II)
            SblkIV.append(S_IV)
            Mdiag.append(M_II)

        # Assemble stiffness matrix
        SblkII = sp.sparse.block_diag(SblkII)
        SblkIV = sp.sparse.vstack(SblkIV)
        M_VV = sp.sparse.dia_matrix(
            (n_adj * he, np.array([0])), shape=(n_nodes, n_nodes))
        D_VV = sp.sparse.dia_matrix(
            (n_adj / he, np.array([0])), shape=(n_nodes, n_nodes))
        SblkVV = M_VV + tau * eps * D_VV
        S = sp.sparse.bmat(
            [[SblkII, SblkIV], [SblkIV.transpose(), SblkVV]], format='csc')

        # Assemble mass matrix
        Mdiag.append(M_VV)
        M = sp.sparse.block_diag(Mdiag)

        Sop = sp.sparse.linalg.splu(S)
        u = np.zeros((n_dofs, nt))


        for k in range(1, nt):

            print("Timestep {:d}/{:d}".format(k, nt))

            # Assemble right-hand side
            # Assemble advection term
            F = np.zeros((n_dofs,))

            uk = u[:, k - 1]

            # test functions in the interior of each edge
            for i in range(n_edges):

                F_ = np.zeros((nxi,))

                # All entries on the edge
                u_ = self.get_u_edge(uk, i)

                # Flux at right and left end of the interval
                F_ += (f(u_[2:]) + f(u_[1:-1])) / 2 \
                    + 0.5 * eta * (u_[1:-1] - u_[2:])
                F_ += - (f(u_[1:-1]) + f(u_[0:-2])) / 2 \
                    + 0.5 * eta * (u_[1:-1] - u_[0:-2])

                # Insert into global flux vector
                F[i * nxi + idxi] = F_

            # test functions in vertices
            for i in range(n_nodes):
                u_v = uk[n_inner + i]

                for j in self.graph.Vout[i]:

                    u_e = uk[j * nxi]
                    F[n_inner + i] += (f(u_v) + f(u_e)) / 2 \
                        + .5 * eta * (u_v - u_e)
                for j in self.graph.Vin[i]:

                    u_e = uk[(j + 1) * nxi - 1]
                    F[n_inner + i] += -(f(u_v) + f(u_e)) / 2 \
                        + .5 * eta * (u_v - u_e)

            for i in self.graph.inflowNodes:
                u_v = uk[n_inner + i]
                F[n_inner + i] += -self.graph.dirichletAlpha[i] * (1 - u_v)

            for i in self.graph.outflowNodes:
                u_v = uk[n_inner + i]
                F[n_inner + i] += self.graph.dirichletBeta[i] * u_v

            # Set up right-hand side
            rhs = M * uk - tau * F

            # Solve equation system
            u[:, k] = Sop.solve(rhs)

        self.u = u

        return u


    
    def write_csv(self, j=0):
        u = self.u
        
        with open('FVM_results.csv', 'w', newline='') as f:
            thewriter = csv.writer(f)
            thewriter.writerow(['spatial', 'temporal', 'result'])
            
            for i in range(u.shape[0]):
                for j in range(u.shape[1]):
                    thewriter.writerow([i, j, u[i,j]])
                
                



Set up constants.

In [3]:
xth_frame = 5
# Time discretization of PINN
N_b = 200
# Time discretization of FVM
nt = N_b * xth_frame

# Spatial discretization of PINN
N_0 = 200
# Spatial discretization of FVM
nx = N_0 + 1

graph = graph.Example3()
graph.buildGraph()

Run FVM solver.

In [4]:
fvm_solver = QuantumGraphSolverFVM(graph)
u = fvm_solver.solve(nx=nx, nt=nt)

Timestep 1/1000
Timestep 2/1000
Timestep 3/1000
Timestep 4/1000
Timestep 5/1000
Timestep 6/1000
Timestep 7/1000
Timestep 8/1000
Timestep 9/1000
Timestep 10/1000
Timestep 11/1000
Timestep 12/1000
Timestep 13/1000
Timestep 14/1000
Timestep 15/1000
Timestep 16/1000
Timestep 17/1000
Timestep 18/1000
Timestep 19/1000
Timestep 20/1000
Timestep 21/1000
Timestep 22/1000
Timestep 23/1000
Timestep 24/1000
Timestep 25/1000
Timestep 26/1000
Timestep 27/1000
Timestep 28/1000
Timestep 29/1000
Timestep 30/1000
Timestep 31/1000
Timestep 32/1000
Timestep 33/1000
Timestep 34/1000
Timestep 35/1000
Timestep 36/1000
Timestep 37/1000
Timestep 38/1000
Timestep 39/1000
Timestep 40/1000
Timestep 41/1000
Timestep 42/1000
Timestep 43/1000
Timestep 44/1000
Timestep 45/1000
Timestep 46/1000
Timestep 47/1000
Timestep 48/1000
Timestep 49/1000
Timestep 50/1000
Timestep 51/1000
Timestep 52/1000
Timestep 53/1000
Timestep 54/1000
Timestep 55/1000
Timestep 56/1000
Timestep 57/1000
Timestep 58/1000
Timestep 59/1000
Timest

Timestep 777/1000
Timestep 778/1000
Timestep 779/1000
Timestep 780/1000
Timestep 781/1000
Timestep 782/1000
Timestep 783/1000
Timestep 784/1000
Timestep 785/1000
Timestep 786/1000
Timestep 787/1000
Timestep 788/1000
Timestep 789/1000
Timestep 790/1000
Timestep 791/1000
Timestep 792/1000
Timestep 793/1000
Timestep 794/1000
Timestep 795/1000
Timestep 796/1000
Timestep 797/1000
Timestep 798/1000
Timestep 799/1000
Timestep 800/1000
Timestep 801/1000
Timestep 802/1000
Timestep 803/1000
Timestep 804/1000
Timestep 805/1000
Timestep 806/1000
Timestep 807/1000
Timestep 808/1000
Timestep 809/1000
Timestep 810/1000
Timestep 811/1000
Timestep 812/1000
Timestep 813/1000
Timestep 814/1000
Timestep 815/1000
Timestep 816/1000
Timestep 817/1000
Timestep 818/1000
Timestep 819/1000
Timestep 820/1000
Timestep 821/1000
Timestep 822/1000
Timestep 823/1000
Timestep 824/1000
Timestep 825/1000
Timestep 826/1000
Timestep 827/1000
Timestep 828/1000
Timestep 829/1000
Timestep 830/1000
Timestep 831/1000
Timestep 8

In [5]:
fvm_solver.write_csv()